In [98]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle

X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

In [99]:
# Pierwszą rzeczą jaką robimi jest normalizacja danych
# Najłatwiej jest to zrobić przez skalowanie danych w obrazie
# Wiemy że najmniejszą liczbą jest 0 a największą 255 (ilość kolorów piksela?)


# Można by użyć czegoś lepszego np. tf.keras.utils.normalize, ale na razie starczy nam prosty sposób
# Albo to co poniżej pewnie też by zadziałało, tylko w innej postaci tablicy
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X = scaler.fit_transform(X)

X = X/255.0

In [100]:
model = Sequential()

# print(X.shape[1:]) # - określa kształt jaki wrzucamy do input_shape (50, 50, 1)
# czyli obraz 50x50, gdzie 1 oznacza obraz czarno-biały; gdyby dać 3 byłby kolorowy

# filters - oznacza rozmiar przestrzeni wyjściowej; Dzieli obraz na 64 okienka (czyli grid 8x8 ??)

# kernel_size - wielkość grida którym będziemy skanować obrazek, czyli grid o wielkości 3x3

model.add(Conv2D(filters = 64, kernel_size = (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu")) # zamiast dodać funkcje aktywacji tutaj można zrobić Conv2D(64, (3, 3), activation='relu')
model.add(MaxPooling2D(pool_size=(2,2)))# wykorzystujemy MaxPooling aby wybraź maksymalną wartość z grida 2x2 i tak wybiera dla wszystkich gridów znajdujących się na obrazku

# kopiujemy taką samą warstwę, tylko w tym przypadku nie potrzebujemy input_shape, który daje się tylko w pierwszej warstwie
model.add(Conv2D(64, (3,3))) # używanie nazw `filters`, `kernel_size` jest opcjonalne
model.add(Activation("relu")) 
model.add(MaxPooling2D(pool_size=(2,2)))

# Dense layer, jest jednowymiarową tablicą (a nie tak jak wyżej kilkuwymiarową talicą)
# dlatego przed użyciem Dense trzba przekształcić 2 wymiarową tablice do 1 wymiaru
model.add(Flatten())

# Przy tym datasecie najprawdopodobniej nawet nie potrzeba tego używać aby model działał dobrze
model.add(Dense(64)) #TODO: czym było Dense?

# w tym przypadku podaje się 1, ponieważ mamy bipolarne dane, czyli 2 klasy
# gdybyśmy mieli na wyniku więcej klas np. 10 trzeba by było podać Dense(10)
model.add(Dense(1)) 
model.add(Activation("sigmoid"))# softmax chyba dla wielu klas??

In [101]:
# Przykład z udemy

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# # 64 3x3 kernels
# model.add(Conv2D(64, (3, 3), activation='relu'))
# # Reduce by taking the max of each 2x2 block
# model.add(MaxPooling2D(pool_size=(2, 2)))
# # Dropout to avoid overfitting
# model.add(Dropout(0.25))
# # Flatten the results to one dimension for passing into our final layer
# model.add(Flatten())
# # A hidden layer to learn with
# model.add(Dense(128, activation='relu'))
# # Another dropout
# model.add(Dropout(0.5))
# # Final categorization from 0-9 with softmax
# model.add(Dense(10, activation='softmax'))

In [89]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 48, 48, 64)        640       
_________________________________________________________________
activation_28 (Activation)   (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 22, 22, 64)        36928     
_________________________________________________________________
activation_29 (Activation)   (None, 22, 22, 64)        0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 11, 11, 64)        0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 7744)              0         
__________

In [90]:
model.compile(loss="binary_crossentropy", #bo tak zaleca dokumentacj keras, dla wielu klas jest categorical_crossentropy
             optimizer="adam", # bo jest najlepszy
             metrics=['accuracy']) # bo tak ??

In [92]:
model.fit(X, y,
         batch_size=32, #ile zdjęć na raz będziemy przetwarzać, najlepiej dać pomiędzy 20 - 200, w zależności jak dużo ma się dataset
         validation_split=0.1, # 10% - ilość danych które nie będą brane do treningu, ale zostaną wykorzystane do sprawdzenia poprawności modelu
         epochs=10 # ilość epok, a epoka jest iteracją w której są zawarte dane X i y
        )


# udemy zamiast validation_fit robiło się validation_data=(test_images, test_labels)

ValueError: Error when checking target: expected activation_30 to have 2 dimensions, but got array with shape (24946, 50, 50, 1)